In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [2]:
!pip install sentence-transformers

In [3]:
import spacy
from sentence_transformers import SentenceTransformer, util
import json

# Load spaCy model and SentenceTransformer model
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Load expertise data from the JSON file
with open('expertise.json', 'r') as json_file:
    expertise_data = json.load(json_file)


def find_similar_expertise(input_expertise, json_file = expertise_data, threshold=0.5):
    # Create a list to store similar expertise in the same category
    similar_expertise_in_category = []

    # Process input expertise with spaCy
    input_doc = nlp(input_expertise)

    # Iterate through expertise categories in the JSON data
    for category in expertise_data.get("expertise", []):
        category_name = category.get("name", "")
        expertise_list = category.get("subOptions", [])

        # Calculate similarity between input expertise and each expertise in the category
        similarities = util.pytorch_cos_sim(
            model.encode([input_expertise], convert_to_tensor=True),
            model.encode(expertise_list, convert_to_tensor=True)
        )[0]

        # Filter results below the threshold
        filtered_results = [(expertise_list[i], similarities[i].item()) for i in range(len(expertise_list)) if similarities[i].item() > threshold]

        # Add the filtered results to the list
        similar_expertise_in_category.extend([{
            'Category': category_name,
            'Expertise': expertise,
            'Similarity': similarity
        } for expertise, similarity in filtered_results])

    # Sort the list by similarity (higher similarity first)
    similar_expertise_in_category.sort(key=lambda x: x['Similarity'], reverse=True)

    return similar_expertise_in_category

# ================================================================================





c:\Users\shubh\OneDrive\Desktop\workspace\for_all\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
all_expertise_htm_data = []
htm_data = pd.read_excel(r"htmldata.xlsx")
for i in htm_data.columns:
    if "Subfield" in i:
        for j  in htm_data[i].dropna():
            all_expertise_htm_data.append(j)
len(all_expertise_htm_data )      

624

In [5]:
## this cell will make a new file of name "data.json"
## use when you have new set of data set. where you have to change the "all_expertise_htm_data" list to your new expertise list

all_expertise_htm_data = set(all_expertise_htm_data)
for i in all_expertise_htm_data:
  i = i.title()
  similar_expertise = find_similar_expertise(i, expertise_data)
  print(i)
  # print(similar_expertise)
  print()
  print()
  k = 1

  if len(similar_expertise)==0:
    new = {}
    new["name"] = i
    new['subOptions'] = [i]
    expertise_data["expertise"].append(new)
  else:
    for entry in similar_expertise:
      if entry["Similarity"] > 0.66:
        print(f"Category: {entry['Category']}, Similar Expertise: {entry['Expertise']}, Similarity: {entry['Similarity']:.4f}")
        if entry["Similarity"] - k == 0:
          k = k-entry["Similarity"]
    if k!=0:
      other_expertise = next(item for item in expertise_data["expertise"] if item["name"] == similar_expertise[0]['Category'])#similar_expertise[0]
      other_expertise["subOptions"].append(i)
    else:
        # other_expertise = next(item for item in expertise_data["expertise"] if item["name"] == "Other")
        # other_expertise["subOptions"].append(i)
      new = {}
      new["name"] = i
      new['subOptions'] = [i]
      expertise_data["expertise"].append(new)
  expertise_data = expertise_data
  
expertise_data
#monitoring and reporting of closure activities.
file_path = "data.json"#change this name 

# Write dictionary to JSON file
with open(file_path, "w") as json_file:
    json.dump(expertise_data, json_file, indent=4)

Physical Chemistry


Category: Chemistry, Similar Expertise: Physical Chemistry, Similarity: 1.0000
Category: Chemistry, Similar Expertise: Analytical Chemistry, Similarity: 0.7707
Category: Chemistry, Similar Expertise: Environmental Chemistry, Similarity: 0.7302
Category: Chemistry, Similar Expertise: Inorganic Chemistry, Similarity: 0.7051
Ecology


Category: Ecology and Conservation, Similar Expertise: Ecology, Similarity: 1.0000
Category: Chemical Engineering, Similar Expertise: Industrial Ecology, Similarity: 0.6918
Biochemistry


Category: Chemistry, Similar Expertise: Biochemistry, Similarity: 1.0000
Genetics


Category: Biology, Similar Expertise: Genetics, Similarity: 1.0000
Water Quality Testing And Analysis


Category: Water Management and Quality, Similar Expertise: Water Quality Testing And Analysis, Similarity: 1.0000
Category: Water Management and Quality, Similar Expertise: Water Quality Monitoring And Analysis, Similarity: 0.9273
Category: Water Management and Quality

KeyboardInterrupt: 

In [ ]:
## use when you have just one expertise to check
i = "" #put the expertise you want to check
similar_expertise = find_similar_expertise(i, expertise_data)
print(i)
# print(similar_expertise)
print()
print()
k = 1
if len(similar_expertise)==0:
  new = {}
  new["name"] = i
  new['subOptions'] = [i]
  expertise_data["expertise"].append(new)
else:
  for entry in similar_expertise:
    if entry["Similarity"] > 0.66:
      print(f"Category: {entry['Category']}, Similar Expertise: {entry['Expertise']}, Similarity: {entry['Similarity']:.4f}")
      if entry["Similarity"] - k == 0:
        k = k-entry["Similarity"]
  if k!=0:
    other_expertise = next(item for item in expertise_data["expertise"] if item["name"] == similar_expertise[0]['Category'])#similar_expertise[0]
    other_expertise["subOptions"].append(i)
  else:
      # other_expertise = next(item for item in expertise_data["expertise"] if item["name"] == "Other")
      # other_expertise["subOptions"].append(i)
    new = {}
    new["name"] = i
    new['subOptions'] = [i]
    expertise_data["expertise"].append(new)
expertise_data = expertise_data